In [53]:
import pandas as pd

In [54]:
from io import StringIO

dataset = 'NIFTY_SEP.txt'
sio = StringIO(open(dataset).read().replace(':','0'))
column_names = ['NIFTY','Date','Time','Open','High','Low','Close','xxx','yyy']
nifty_data = pd.read_csv(sio, names=column_names)
del nifty_data['NIFTY']
del nifty_data['xxx']
del nifty_data['yyy']

In [55]:
#print(nifty_data.to_string())

In [56]:
import numpy as np
stock_open = np.zeros(264,dtype=float)
stock_close = np.zeros(264,dtype=float)
stock_high = np.zeros(264,dtype=float)
#stock_low = np.zeros(264,dtype=float)
stock_low = np.full(shape=264,fill_value=15000,dtype=float)

In [57]:
#print(type(nifty_data.Date[1]))

In [58]:
time = np.array([9031,10001,10031,11001,11031,12001,12031,13001,13031,14001,14031,15001])
a = 0
for i in range(0,22):
    for j in range(0,12): 
        for x in range(a,10000):
            if (nifty_data.Time[x] == time[j]):
                stock_open[j+(i*12)] = nifty_data.Open[x]
                #print(i,j,x)
                a = x
                break


In [59]:
time = np.array([10000,10030,11000,11030,12000,12030,13000,13030,14000,14030,15000,15030])
a = 0
for i in range(0,22):
    for j in range(0,12):
        for x in range(a,10000):
            if (nifty_data.Time[x] == time[j]):
                stock_close[j+(i*12)] = nifty_data.Close[x] 
                a = x
                break

In [60]:
time = np.array([9031,10001,10031,11001,11031,12001,12031,13001,13031,14001,14031,15001,15030])
a = 0
for i in range(0,22):
    for j in range(0,12): 
        for x in range(a,10000):
            if (nifty_data.Time[x] >= time[j] and nifty_data.Time[x] < time[j+1]):
                stock_high[j+(i*12)] = max(stock_high[j+(i*12)],nifty_data.High[x])
                stock_low[j+(i*12)] = min(stock_low[j+(i*12)],nifty_data.Low[x])
            elif(nifty_data.Time[x] == time[j+1]):
                #print(i,j,x)
                a=x
                break

In [61]:
dictionary = {
    'open':stock_open,
    'high':stock_high,
    'low':stock_low,
    'close':stock_close
}

In [62]:
df = pd.DataFrame(dictionary)
pd.set_option("display.max_rows", None, "display.max_columns", None)
#print(df)

In [63]:
def check_spinning_top(o,h,l,c):
    d = abs(o-c) < (h+l)/5000
    f = abs(((o+c)-(h+l))/2) < (h+l)/5000
    if d and f:
        return 1
    else:
        return 0

In [65]:
def before_trend(i):
    a = (df.high[i] + df.low[i])/2
    a_5 = (df.high[i-5] + df.low[i-5])/2
    if a > a_5:
        return 1
    else:
        return 0

In [66]:
#before_trend(14)

In [67]:
def after_trend(i):
    a = (df.high[i] + df.low[i])/2
    a_2 = (df.high[i+2] + df.low[i+2])/2
    if a < a_2:
        return 1
    else:
        return 0

In [68]:
#after_trend(14)

In [69]:
def trading():
    total_shares = 0
    total_money = 500000
    final_spinning_top = 0
    for x in df.index:
        if(check_spinning_top(df.open[x],df.high[x],df.low[x],df.close[x])):
            #print(df.open[x],df.high[x],df.low[x],df.close[x])
            #print(x)
            if(before_trend(x)):
                total_money = total_money + (total_shares/2)*(df.close[x])
                total_shares = total_shares/2
                if(after_trend(x)):
                    total_money = total_money - (total_shares)*(df.close[x+2])
                    total_shares = 2*total_shares
                else:
                    total_money = total_money + (total_shares)*(df.close[x+2])
                    total_shares = 0
            else:
                total_money = total_money - 10*(df.close[x])
                total_shares = total_shares + 10
                if(after_trend(x)):
                    total_money = total_money - 10*(df.close[x+2])
                    total_shares = total_shares + 10
                else:
                    total_money = total_money + 10*(df.close[x+2])
                    total_shares = total_shares - 10
            final_spinning_top = x        
    total_money = total_money + (total_shares*(df.close[final_spinning_top]))
    total_shares = 0
    print(total_money,total_shares)
trading()    

495971.5 0
